In [15]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
from gensim.models import Word2Vec

In [16]:
def build_sent_graph(tfidf, sentences):
    tfidf_mat = tfidf.fit_transform(sentences).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

def build_words_graph(cnt_vec, sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [17]:
def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    is_zrop_arr = []
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        else:
            is_zrop_arr.append(id)

    for id in is_zrop_arr:
        A[id, :] = 0
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}
def summarize(sorted_sent_rank_idx, texts, sent_num=3):
    summary = []
    index=[]
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
    index.sort()
    for idx in index:
        summary.append(texts[idx])
    return summary
def keywords(words_graph, sorted_rank_idx, idx2word, word_num=10):
    rank_idx = get_ranks(words_graph)
    sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
    keywords = []
    index=[]
    for idx in sorted_rank_idx[:word_num]:
        index.append(idx)
    
    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])
    return keywords

In [18]:
stopwords = sorted(['동안', '방안', '상주', '제시', '구체적', '1항', '민사', 
                    '과의', '사용', '개월', '디지털트윈', '경우', '관련', '함', '디지털', '직원', '고려', '가능',
                    '등에', '모든', '보장', '국가시범도시', '변경', '무엇', '동일', '물론', '지가', '대시', '번호', '도시', '다음'])
def noun_condition(noun):
    return noun in stopwords 

import re

def run(texts):
    
    kkma = Kkma()

    nouns = []
    for sentence in texts:
        if sentence is not '':
            noun_unit_arr = []
            noun_arr = kkma.nouns(sentence)
            for noun in noun_arr:
                if noun_condition(noun):
                    continue
                elif noun.isdigit():
                    continue
                elif len(re.findall('[0-9]', noun)) != 0:
                    continue
                else:
                    noun_unit_arr.append(noun)
            nouns.append(' '.join(noun_unit_arr))
    tfidf = TfidfVectorizer()
    cnt_vec = CountVectorizer()

    sent_graph = build_sent_graph(tfidf, nouns)
    words_graph, idx2word = build_words_graph(cnt_vec, nouns)

    sent_rank_idx = get_ranks(sent_graph)
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)
    word_rank_idx = get_ranks(words_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
    summarize_arr = summarize(sorted_sent_rank_idx, texts, 3)
    keyword_arr = keywords(words_graph, sorted_word_rank_idx, idx2word, word_num=10)
    
    return (summarize_arr, keyword_arr)



### 문장에 대한 최종 핵심 요약정보 정리 표출 

In [35]:
### summerize 정보 구축
texts = ['제안업체 기술의 적합성, 자질 및 활용방안을 입증할 수 있는 증빙자료를 제시하여야 함',
'공동수급 및 하도급 구성시 참여 기업별로 수행범위 및 역할, 상호 협력 방안 등을 구체적이고 명확하게 정의하고 조직도를 제시하여야 함',
'제안사는 사업의 안정적이고 효율적인 시스템 구축을 위해 사업수행 책임자(PM)는 주사업자의 직원으로 본 사업을 총괄하여야 하며, 사업 전 기간 동안 상주하여야 함(상주 장소는 발주기관과 협의)',
'인력이 교체되는 경우 업무의 안정화 및 품질저하 방지대책을 협의하여야 함']
print(texts)
arr_text = []


import json # import json module

arr_text.append(texts)
# with statement
with open('C://Deport/2020_RFPSEARCH/rfpquicksearch/rs-client/db.json', encoding="utf8") as json_file:
    json_data = json.load(json_file)
    json_string = json_data["rfp"]
    for obj in json_string:
        arr_text.append([obj['detInfo']])
arr_three_summary = []
for p in arr_text:
    a, b = run(p)
    print(b)
    arr_three_summary.append(b)

['제안업체 기술의 적합성, 자질 및 활용방안을 입증할 수 있는 증빙자료를 제시하여야 함', '공동수급 및 하도급 구성시 참여 기업별로 수행범위 및 역할, 상호 협력 방안 등을 구체적이고 명확하게 정의하고 조직도를 제시하여야 함', '제안사는 사업의 안정적이고 효율적인 시스템 구축을 위해 사업수행 책임자(PM)는 주사업자의 직원으로 본 사업을 총괄하여야 하며, 사업 전 기간 동안 상주하여야 함(상주 장소는 발주기관과 협의)', '인력이 교체되는 경우 업무의 안정화 및 품질저하 방지대책을 협의하여야 함']
['수행', '제안', '협의', '발주기관', '장소', '기간', '사업', '사업수행', '안정적', '효율적']
['갱신', '방식', '데이터', '설정', '범위', '송수신', '연계', '운용', '재개발', '기술']
['상황이벤트', '성능', '교환', '구성', '네트워크', '복합', '이벤트', '발생', '성시', '이슈']
['가능성', '공간', '기관', '결과', '기본', '공개', '공공기관', '대용량', '수시', '업무활용기관']
['공공정보', '구축', '공공', '구성', '구현시', '기술', '정보', '트윈', '방식', '인터페이스']
['계획', '공간', '관리운영', '기반데이터', '기반', '구성', '기반정보', '구축', '대표', '도시계획']
['개선', '공통', '구축', '구성', '메뉴', '공개', '관점', '구현', '기능', '독자적']
['각종', '개발', '미비로', '불가능', '사유', '안전', '의무', '지침', '브라우저', '조정']
['단위', '레벨', '공통활용', '관리', '구분', '기준', '이력', '기능', '모듈', '이력관리']
['결과', '국토', '구분', '결함', '계약', '과제', '규정', '기록', '국정원', '보고서']
['도출', '수립', '보안', '보안점검', '개선', '계획', '상대자', '수행', '점검'

In [34]:
print(len(arr_three_summary))
print(arr_three_summary)

92
[['수행', '제안', '협의', '발주기관', '장소', '기간', '사업', '사업수행', '안정적', '효율적'], ['갱신', '방식', '데이터', '설정', '범위', '송수신', '연계', '운용', '재개발', '기술'], ['상황이벤트', '성능', '교환', '구성', '네트워크', '복합', '이벤트', '발생', '성시', '이슈'], ['가능성', '공간', '기관', '결과', '기본', '공개', '공공기관', '대용량', '수시', '업무활용기관'], ['공공정보', '구축', '공공', '구성', '구현시', '기술', '정보', '트윈', '방식', '인터페이스'], ['계획', '공간', '관리운영', '기반데이터', '기반', '구성', '기반정보', '구축', '대표', '도시계획'], ['개선', '공통', '구축', '구성', '메뉴', '공개', '관점', '구현', '기능', '독자적'], ['각종', '개발', '미비로', '불가능', '사유', '안전', '의무', '지침', '브라우저', '조정'], ['단위', '레벨', '공통활용', '관리', '구분', '기준', '이력', '기능', '모듈', '이력관리'], ['결과', '국토', '구분', '결함', '계약', '과제', '규정', '기록', '국정원', '보고서'], ['도출', '수립', '보안', '보안점검', '개선', '계획', '상대자', '수행', '점검', '제반'], ['권한', '공간', '기능', '내부', '등록자', '부여', '사용자', '공간정보포탈', '등록', '로그인'], ['로그', '백업', '시스템별', '구축', '수정', '시스템', '유지', '이상', '최소'], ['국가', '대응', '발생', '사이버', '안전', '안전매뉴얼', '절차', '준수', '침해', '침해사고'], ['과업수행', '계약', '국가', '누출', '당사자', '무단', '불가', '사업', '사업수행자', '손해']

In [23]:
## PreTrain Topick 모델

from gensim import corpora
dictionary = corpora.Dictionary(arr_three_summary)
corpus = [dictionary.doc2bow(text) for text in arr_three_summary]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0
print(len(dictionary))

[(10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]
511


In [24]:
## Iteration에 따른 변화 확인
num_topics = 10
chunksize = 2000
iterations = 400
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel


In [25]:
# 최적의 Iteration 개수를 찾는다 -- not use
coherences=[]
perplexities=[]
passes=[]
eval_every = None
for i in range(10):
    ntopics, nwords = 200, 100
    if i==0:
        passes=1
    else:
        passes=i*5
    model = LdaModel(corpus = corpus, id2word = dictionary, chunksize = chunksize,\
                           alpha ="auto", eta="auto",\
                           iterations = iterations, num_topics = num_topics,\
                           passes = passes, eval_every = eval_every)
    
    cm = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print(passes)
    print("Cpherence",coherence)
    print('Perplexity: ', model.log_perplexity(corpus), '\n')

1
Cpherence -16.560277109610702
Perplexity:  -6.885745142837387 

5
Cpherence -16.79392206669034
Perplexity:  -6.764268138416461 

10
Cpherence -16.73858685021108
Perplexity:  -6.7257535058188465 

15
Cpherence -16.453677582484165
Perplexity:  -6.735267866008805 

20
Cpherence -15.824611331250555
Perplexity:  -6.752417019305338 

25
Cpherence -15.546671292554084
Perplexity:  -6.7095852985961875 

30
Cpherence -15.680880531428206
Perplexity:  -6.722066321153287 

35
Cpherence -15.83501171596303
Perplexity:  -6.70510301403417 

40
Cpherence -16.41161452305941
Perplexity:  -6.709424813935125 

45
Cpherence -15.918099159893817
Perplexity:  -6.700529994497902 



In [93]:
# 최적의 Topic 개수를 찾는다 -- not use
coherencesT=[]
perplexitiesT=[]
passes=[]
ntopics = 2
for i in range(10):
    if i == 0:
        ntopics = 2
    else:
        ntopics *= 2
    model = LdaModel(corpus = corpus, id2word = dictionary, chunksize = chunksize,\
                           alpha ="auto", eta="auto",\
                           iterations = iterations, num_topics = nwords,\
                           passes = 20, eval_every = eval_every)
    
    cm = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print(ntopics)
    print("Cpherence",coherence)
    print('Perplexity: ', model.log_perplexity(corpus), '\n')


2
Cpherence -17.304710644043798
Perplexity:  -7.2086942454340655 

4
Cpherence -17.329241255820182
Perplexity:  -7.289571599424047 

8
Cpherence -17.311937537673685
Perplexity:  -7.220144536921649 

16
Cpherence -17.28629629159504
Perplexity:  -7.2340080951287495 

32
Cpherence -17.34523689965144
Perplexity:  -7.220630607953885 

64
Cpherence -17.276315609696663
Perplexity:  -7.268262670776708 

128
Cpherence -17.287751504816978
Perplexity:  -7.145537768760671 

256
Cpherence -17.226062125685676
Perplexity:  -7.208880379755645 

512
Cpherence -17.281422352952973
Perplexity:  -7.271008191554527 

1024
Cpherence -17.2569251422303
Perplexity:  -7.208360928345502 



In [31]:
import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, alpha='auto', eval_every=5, id2word=dictionary, passes=100)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)


(0, '0.016*"발주기관" + 0.016*"사업" + 0.016*"내용" + 0.016*"계약"')
(1, '0.025*"관리" + 0.015*"사항" + 0.015*"기능" + 0.015*"사업"')
(2, '0.026*"관리" + 0.017*"정기" + 0.017*"가용성" + 0.017*"경찰청"')
(3, '0.021*"기술" + 0.021*"방법" + 0.016*"데이터" + 0.016*"수행"')
(4, '0.021*"제안" + 0.021*"사업" + 0.011*"등록" + 0.011*"로그인"')
(5, '0.034*"시스템" + 0.029*"구축" + 0.024*"사업" + 0.020*"관리"')
(6, '0.026*"데이터" + 0.019*"관리" + 0.013*"보안" + 0.013*"각종"')
(7, '0.021*"구성" + 0.021*"검증" + 0.011*"변경관리" + 0.011*"자동화"')
(8, '0.021*"수집" + 0.021*"데이터" + 0.011*"법령상의" + 0.011*"보호법"')
(9, '0.028*"서비스" + 0.019*"교체" + 0.019*"관리" + 0.019*"사업"')


In [27]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\Daumsoft\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\Daumsoft\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [28]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(1, 0.99109143)]
1 번째 문서의 topic 비율은 [(4, 0.99269986)]
2 번째 문서의 topic 비율은 [(3, 0.99059457)]
3 번째 문서의 topic 비율은 [(4, 0.9926999)]
4 번째 문서의 topic 비율은 [(0, 0.9919127)]


In [29]:
import pandas as pd
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)


In [30]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,1.0,0.9911,"[(1, 0.99109143)]"
1,1,4.0,0.9927,"[(4, 0.99269986)]"
2,2,3.0,0.9906,"[(3, 0.99059457)]"
3,3,4.0,0.9927,"[(4, 0.9926999)]"
4,4,0.0,0.9919,"[(0, 0.9919127)]"
5,5,0.0,0.9919,"[(0, 0.99191266)]"
6,6,0.0,0.9919,"[(0, 0.99191266)]"
7,7,3.0,0.9906,"[(3, 0.9905945)]"
8,8,0.0,0.9919,"[(0, 0.9919127)]"
9,9,0.0,0.9919,"[(0, 0.99191266)]"
